## Everything In Python Is An Object

Everything in Python is an object, and almost everything has attributes and methods. All functions have a built-in attribute `__doc__`, which returns the doc string defined in the function's source code. The `sys` module is an object which has (among other things) an attribute called `path`. And so forth.

P.S. In old versions of Python the concepts of "class" and "type" are not related. `type(obj)` returns confusing `<type 'instance'>` value; we have to call `obj.__class__` when we need to know the object class.

[The Inside Story on New-Style Classes](http://python-history.blogspot.com/2010/06/inside-story-on-new-style-classes.html)

---

_An object is a unit of data_ (having one or more attributes), of a particular _class_ or _type_, with associated functionality (methods).

* _class_ – a blueprint for an instance
* _instance_ – a constructed _object_ of the class
* _attribute_ – any object value
* _method_ – a "callable attribute" defined in the class. In other words, it is a function that is stored as a class attribute.

In [1]:
type(None)

NoneType

In [2]:
type(type('omg!'))

type

In [3]:
dir(5)

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes']

## Classes

Overall syntax:

```python
class SomeClass(Base1, Base2, ...):
    """Useful documentation."""
    class_attr = ...
    
    def __init__(self, some_arg):
        self.instance_attr = some_arg
        
    def some_method(self):
        pass
```

`self` is an explicit reference to a class instance.

Example:

In [4]:
class Counter:
    """I count. That is all."""
    def __init__(self, initial=0):  # constructor
        self.value = initial        # object attribute
        
    def increment(self):
        self.value += 1
        
    def get(self):
        return self.value

In [5]:
c = Counter(42)

In [6]:
c.increment()

In [7]:
c.get()

43

## "Private" Methods and Attributes

There are no such things in Python. Everything is public. Developers just follow naming conventions to inform about internal class members.

In [8]:
class Noop:
    some_attribute = 42
    _interal_attribute = []  # name of the attribute starts with _

In [9]:
Noop._interal_attribute

[]

Some programmers use `__` (double underscore) instead of `_` to really emphasize the purpose.

In [10]:
class Noop:
    __internal_attribute = []

In [11]:
Noop.__internal_attribute

AttributeError: type object 'Noop' has no attribute '__internal_attribute'

But a class member is still publicly accessible. I don't think it worth it.

In [12]:
Noop._Noop__internal_attribute

[]

## Class Attributes

⚠️  Only for educational purposes. This class is really broken.

In [13]:
from collections import deque


class MemorizingDict(dict):
    _history = deque(maxlen=10)  # class attribute
    
    def set(self, key, value):
        self._history.append(key)
        self[key] = value
        
    def get_history(self):
        return self._history

In [14]:
d = MemorizingDict({"foo": 42})
d.set("baz", 100500)
d.get_history()

deque(['baz'])

In [15]:
d = MemorizingDict()
d.set("boo", 500100)
d.get_history()

deque(['baz', 'boo'])

## Magic Methods

In [16]:
class Noop:
    """I do nothing at all."""

In [17]:
Noop.__doc__

'I do nothing at all.'

In [18]:
Noop.__name__

'Noop'

In [19]:
Noop.__base__

object

In [20]:
Noop.__module__

'__main__'

In [21]:
noop = Noop()
noop.__class__

__main__.Noop

In [22]:
noop.__dict__  # all attributes of an object

{}

In [23]:
# Class is also an object
Noop.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'I do nothing at all.',
              '__dict__': <attribute '__dict__' of 'Noop' objects>,
              '__weakref__': <attribute '__weakref__' of 'Noop' objects>})

In [24]:
Noop.__class__

type

ℹ️ _We can work with object or class attributes in the same way we work with a dictionary._

In [25]:
noop.some_attribute = 42
noop.__dict__

{'some_attribute': 42}

In [26]:
# just for example
noop.__dict__["some_attribute"] = 100500
noop.some_attribute

100500

In [27]:
del noop.some_attribute

In [28]:
noop.__dict__

{}

🔮 Some dark magic:

In [29]:
noop.number = 42
noop.string = "Hello"
noop.__dict__

{'number': 42, 'string': 'Hello'}

In [30]:
del noop.__dict__
noop.__dict__

{}

Attribute lookup happens dynamically at runtime.

In [31]:
vars(noop)  # noop.__dict__

{}

## Bind Methods

In [32]:
class SomeClass:
    def do_something(self):
        print("Doing something.")

In [33]:
SomeClass().do_something  # method is bind to the class instance

<bound method SomeClass.do_something of <__main__.SomeClass object at 0x108223890>>

In [34]:
SomeClass().do_something()

Doing something.


In [35]:
SomeClass.do_something  # unbind method is just a function

<function __main__.SomeClass.do_something(self)>

In [36]:
instance = SomeClass()

In [37]:
SomeClass.do_something(instance)

Doing something.


## Properties

Properties are useful when we want to **evaluate** them **during a call**.

In [38]:
from os.path import dirname


class Path:
    def __init__(self, current):
        self.current = current
        
    def __repr__(self):
        return f"Path({self.current})"
    
    @property
    def parent(self):
        return Path(dirname(self.current))  # evaluate at every call

In [39]:
p = Path("/Users/akrisanov/.zshrc")

In [40]:
p.parent

Path(/Users/akrisanov)

### Getters, Setters, Deleters

In [41]:
class BigDataModel:
    def __init__(self):
        self._params = []
        
    @property
    def params(self):
        return self._params
    
    @params.setter
    def params(self, new_params):
        assert all(map(lambda p: p > 0, new_params))
        self._params = new_params
        
    @params.deleter
    def params(self):
        del self._params

In [42]:
model = BigDataModel()
model.params = [0.1, 0.5, 0.4]
model.params

[0.1, 0.5, 0.4]

## Inheritance

In [43]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial

In [44]:
class OtherCounter(Counter):
    def get(self):
        return self.value

In [45]:
c = OtherCounter()  # Counter.__init__
c.get()             # OtherCounter.get()

0

In [46]:
c.value            # c.__dict__["value"]

0

### super()

[Documentation](https://docs.python.org/3/library/functions.html#super)

In [47]:
class Counter:
    all_counters = []
    
    def __init__(self, initial=0):
        self.__class__.all_counters.append(self)
        self.value = initial

In [48]:
class OtherCounter(Counter):
    def __init__(self, initial=0):
        self.initial = initial
        super().__init__(initial)

In [49]:
oc = OtherCounter()
vars(oc)

{'initial': 0, 'value': 0}

### Predicates

In [50]:
class A:
    pass


class B(A):
    pass

In [51]:
isinstance(B(), A)

True

In [52]:
class C:
    pass

In [53]:
isinstance(B(), (A, C))

True

In [54]:
isinstance(B(), A) or isinstance(B(), C)

True

In [55]:
# Why not just write type(B()) == A?
type(B())

__main__.B

### Multiple Inheritance

[C3 linearization](https://en.wikipedia.org/wiki/C3_linearization)

[The C3 algorithm, under control of a total order](http://doc.sagemath.org/html/en/reference/misc/sage/misc/c3_controlled.html#)

In [56]:
class A:
    def f(self):
        print("A.f")
        
        
class B:
    def f(self):
        print("B.f")
        
        
class C(A, B):
    pass

In [57]:
C().f()

A.f


In [58]:
C.mro()  # list(C.__mro__)

[__main__.C, __main__.A, __main__.B, object]

### Mixins

[Wikipedia](https://en.wikipedia.org/wiki/Mixin)

## Class Decorators

[PEP 3129](https://www.python.org/dev/peps/pep-3129/)

Class decorators work in the same way as function decorators, but they act on classes rather than functions.

```python
@deco
class Noop:
    pass


# vs

class Noop:
    pass


Noop = deco(Noop)
```

In this case, a decorator is a function that takes a class as an argument and returns another (modified) class.

We can use class decorators instead of using mixins.

### Singleton

In [59]:
import functools


def singleton(cls):
    instance = None
    
    @functools.wraps(cls)
    def inner(*args, **kwargs):
        nonlocal instance
        if instance is None:
            instance = cls(*args, **kwargs)
        return instance
    
    return inner

ℹ️ _`wraps` also copies `__dict__` from the decorated class._

In [60]:
@singleton
class Noop:
    """I do nothing at all"""

In [61]:
id(Noop())

4431558288

In [62]:
id(Noop())

4431558288

### Deprecation Warning

In [63]:
import warnings

In [64]:
def deprecated(cls):
    orig_init = cls.__init__
    
    @functools.wraps(cls.__init__)
    def new_init(self, *args, **kwargs):
        warnings.warn(
            cls.__name__ + " is deprecated.",
            category=DeprecationWarning)
        orig_init(self, *args, **kwargs)
        
    cls.__init__ = new_init
    return cls

In [65]:
@deprecated
class Counter:
    def __init__(self, initial=0):
        self.value = initial

In [66]:
c = Counter()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Counter is deprecated.
  


## Magic Methods

[Special method names](https://docs.python.org/3/reference/datamodel.html#special-method-names)

### `__getattr__`

In [67]:
class Noop:
    pass

In [68]:
Noop().foobar

AttributeError: 'Noop' object has no attribute 'foobar'

In [69]:
class Noop:
    def __getattr__(self, name):
        return name # identity

In [70]:
Noop().foobar

'foobar'

### `__setattr__` and `__delattr__`

Unlike `__getattr__` these methods are called for all attributes, even existing ones.

In [71]:
# Just for educational purposes!

class Guarded:
    guarded = []
    
    def __setattr__(self, name, value):
        assert name not in self.guarded
        super().__setattr__(name, value)

In [72]:
class Noop(Guarded):
    guarded = ["foobar"]
    
    def __init__(self):
        self.__dict__["foobar"] = 42  # skip __setattr__ method call

Also check out:

- [Bunch library](https://github.com/dsc/bunch)
- [Packt: Bunch](https://subscription.packtpub.com/book/application_development/9781788830829/1/ch01lvl1sec18/bunch)

In [73]:
class Noop:
    some_attribute = 42

In [74]:
noop = Noop()
getattr(noop, "some_attribute")

42

In [75]:
getattr(noop, "some_other_attribute", 100500)

100500

In [76]:
setattr(noop, "some_other_attribute", 100500)

In [77]:
delattr(noop, "some_other_attribute")

### Comparison Methods

```python
instance.__eq__(other)  # instance == other
instance.__ne__(other)  # instance != other
instance.__lt__(other)  # instance < other
instance.__le__(other)  # instance <= other
instance.__gt__(other)  # instance > other
instance.__ge__(other)  # instance >= other
```

In [78]:
import functools


@functools.total_ordering  # <- helps a lot, but it's not that fast
class Counter:
    def __eq__(self, other):  # required
        return self.value == other.value
    
    def __lt__(self, other):  # or <=, >, >=
        return self.value < other.value

### `__call__`

In [79]:
class Identity:
    def __call__(self, x):
        return x

In [80]:
Identity()(42)

42

Usecase?

**Class-based decorator with arguments:**

Sometimes it can be useful and help readability. The initialization and the decorator logic are separated.

💡 When you have a long function-base decorator, think about class-based decorator.

In [81]:
class trace:
    def __init__(self, handle):
        self.handle = handle
        
    def __call__(self, func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs, file=self.handle)
            return func(*args, **kwargs)
        return inner

In [82]:
import sys


@trace(sys.stderr)
def identity(x):
    return x

In [83]:
identity(42)

identity (42,) {}


42

### Conversion to String

In [84]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
    def __repr__(self):
        return f"Counter({self.value})"
    
    def __str__(self):
        return f"Counter to {self.value}"

c = Counter(42)

In [85]:
c  # __repr__

In [86]:
print(c)  # __str__

### __format__

In [87]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
    def __format__(self, format_spec):
        return self.value.__format__(format_spec)

In [88]:
c = Counter(42)

In [89]:
"Counted to {:b}".format(c)

'Counted to 101010'

### __hash__

```python
x is y <=> hash(x) == hash(y)
```

Couple advice:

- Implement `__hash__` method together with `__eq__`
    - `__hash__` must satisfy `x == y => hash(x) == hash(y)`
- For mutable objects, it's ok to define only `__eq__` method

### __bool__

Helps to check for truthy / falsy.

In [90]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
    def __bool__(self):
        return bool(self.value)

In [91]:
c = Counter()

In [92]:
if not c:
    print("No counts yet.")

No counts yet.


## Descriptors

- [Python Descriptors Demystified](https://nbviewer.jupyter.org/urls/gist.github.com/ChrisBeaumont/5758381/raw/descriptor_writeup.ipynb)
- [Python Descriptors: An Introduction](https://realpython.com/python-descriptors/)
- [Descriptor HowTo Guide](https://docs.python.org/3/howto/descriptor.html)
- [cached-property: Don't copy/paste code](https://www.pydanny.com/cached-property.html)

> Python descriptors are a way to create managed attributes. Among their many advantages, managed attributes are used to protect an attribute from changes or to automatically update the values of a dependant attribute.

Semantic of the descriptor:

```
cls.attr                 descr.__get__(None, cls)
instance.attr          ≃ descr.__get__(instance, cls)
instance.attr = value    descr.__set__(instance, value)
del instance.attr        descr.__delete__(instance)
```

Types of descriptors:

- _data descriptors_ – implement at least the `__set__` method
- _non-data descriptors_

Example:

```python
# Descriptor class
class NonNegative:
    def __get__(self, instance, owner):
        return magically_get_value(...)
    
    def __set__(self, instance, value):
        assert value >= 0, "non-negative value required"
        magically_set_value(...)
        
    def __delete__(self, instance):
        magically_delete_value(...)
        

# Our class
class Point:
    x = NonNegative()
    y = NonNegative()
    
    
# Some logic
p = Point()
p.x = 42
p.x = - 42  #  AssertError: non-negative value required
p.y = 0
p.y = -1  #  AssertError: non-negative value required
```

### `__get__` and _owner_

In [1]:
class Descr:
    def __get__(self, instance, owner):
        print(instance, owner)

In [2]:
class A:
    attr = Descr()

In [3]:
A.attr

None <class '__main__.A'>


In [4]:
A().attr

<__main__.A object at 0x1096ebf10> <class '__main__.A'>


In [5]:
class B(A):
    pass

In [6]:
B.attr

None <class '__main__.B'>


In [7]:
B().attr

<__main__.B object at 0x1096c4710> <class '__main__.B'>


### `__set__`

In [8]:
class Descr:
    def __set__(self, instance, value):
        print(instance, value)

In [9]:
class A:
    attr = Descr()

In [10]:
instance = A()
instance.attr = 42

<__main__.A object at 0x109af22d0> 42


In [11]:
A.attr = 42  # boom!

In [12]:
A.attr

42

### `__delete__`

In [20]:
class Descr:
    def __delete__(self, instance):
        print(instance)

In [21]:
class A:
    attr = Descr()

In [22]:
A.__dict__["attr"]  # descriptor

In [23]:
del A().attr

In [24]:
del A.attr

⚠️ P.S. `__delete__` is called to delete the attribute on an instance instance of the owner class. `__del__` is a destructor method which is called as soon as all references of the object are deleted i.e when an object is garbage collected.

### Data-descriptor

In [25]:
class Descr:
    def __get__(self, instance, owner):
        print("Descr.__get__")
        
    def __set__(self, instance, value):
        print("Descr.__set__")

In [27]:
class A:
    attr = Descr()

In [28]:
instance = A()

In [29]:
instance.attr

Descr.__get__


In [31]:
instance.__dict__["attr"] = 42  # skip the __set__ method

In [32]:
instance.attr

Descr.__get__


### Descriptor with the `__get__` method

In [33]:
class Descr:
    def __get__(self, instance, owner):
        print("Descr.__get__")

In [34]:
class A:
    attr = Descr()

In [35]:
instance = A()
instance.attr

Descr.__get__


In [36]:
instance.__dict__["attr"] = 42  # or instance.attr = 42

In [37]:
instance.attr

42

### `@property`

In [38]:
class property:
    def __init__(self, get=None, set=None, delete=None):
        self._get = get
        self._set = set
        self._delete = delete
        
    def __get__(self, instance, owner):
        if self._get is None:
            raise AttributeError("unreadable attribute")
        return self._get(instance)
    
    # ...
    # __set__
    # __delete__

In [39]:
class Something:
    @property
    def attr(self):
        return 42

### Class Methods and Descriptors

In [40]:
class Something:
    def do_something(self):
        pass

In [41]:
Something.do_something

<function __main__.Something.do_something(self)>

In [42]:
Something().do_something

<bound method Something.do_something of <__main__.Something object at 0x109be6bd0>>

So, how it works? Descriptors!

In [43]:
from types import MethodType


class Function:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return MethodType(self, instance, owner)

### `@staticmethod` and `@classmethod`

In [51]:
class SomeClass:
    @staticmethod
    def do_something():
        pass

In [52]:
SomeClass.do_something()

In [53]:
SomeClass().do_something()

In [54]:
class Settings:
    @classmethod
    def read_from(cls, path):
        return cls()  # noop

In [55]:
Settings.read_from("./settings.ini")

In [56]:
class staticmethod:
    def __init__(self, method):
        self._method = method
        
    def __get__(self, instance, owner):
        return self._method

In [58]:
import functools


class classmethod:
    def __init__(self, method):
        self._method = method
        
    def __get__(self, instance, owner):
        return functools.partial(self._method, owner)

## Metaclasses

All classes in Python are instances of the class `type`.

[PEP 3115 -- Metaclasses in Python 3000](https://www.python.org/dev/peps/pep-3115/)

In [59]:
class Something:
    attr = 42

In [60]:
Something

__main__.Something

In [61]:
type(Something)

type

The `type` class is called a _metaclass_. Instances of this class are also classes.

Let's create a new class `Something`:

In [62]:
name, bases, attrs = "Something", (), {"attr": 42}

In [63]:
Something = type(name, bases, attrs)

In [64]:
Something

__main__.Something

In [65]:
some = Something()

In [66]:
some.attr

42

### Using Custom Metaclass

In [67]:
class Meta(type):
    def some_method(cls):
        return "foobar"

In [68]:
class Something(metaclass=Meta):
    attr = 42

In [69]:
type(Something)

__main__.Meta

In [70]:
Something.some_method

<bound method Meta.some_method of <class '__main__.Something'>>

In [71]:
Something().some_method

AttributeError: 'Something' object has no attribute 'some_method'

### `__new__` magic method

- The `__new__` method creates a new class instance
- The `__init__` method initializes created instance

In [73]:
class Noop:
    def __new__(cls, *args, **kwargs):
        print(f"Creating instance with {args} and {kwargs}")
        instance = super().__new__(cls)  # self
        return instance
    
    def __init__(self, *args, **kwargs):
        print(f"Initializing with {args} and {kwargs}")

In [74]:
noop = Noop(42, attr="value")

Creating instance with (42,) and {'attr': 'value'}
Initializing with (42,) and {'attr': 'value'}


### Useful Metaclasses

- enum.EnumMeta
- abc.ABCMeta

### Useless Metaclass

In [79]:
from collections import OrderedDict


class UselessMeta(type):
    def __new__(metacls, name, bases, clsdict):
        print(type(clsdict))
        print(list(clsdict))
        cls = super().__new__(metacls, name, bases, clsdict)
        return cls
    
    @classmethod
    def __prepare__(metacls, name, bases):
        return OrderedDict()

In [80]:
class Something(metaclass=UselessMeta):
    attr = "foo"
    other_attr = "bar"

<class 'collections.OrderedDict'>
['__module__', '__qualname__', 'attr', 'other_attr']


---

In [81]:
class A(metaclass=lambda *args, **kwargs: 42): pass

In [82]:
A

42

### Metaclasses vs Class Decorators

- Both can be useful when we want to change class behavior
- But, _metaclasses_:
    - can _temporary_  replace type of `__dict__`
    - are still available after inheritance

In [83]:
class Base(metaclass=Meta):
    pass

In [84]:
class Something(Base):
    pass

In [85]:
type(Something)

__main__.Meta

### Should I Use Metaclasses?

❌ In most cases not!

### Where Metaclasses Can Be Useful?

- ORMs
- Dynamic forms
- Web frameworks

### The `abc` Module

[abc — Abstract Base Classes](https://docs.python.org/3/library/abc.html)

Metaclass `ABCMeta` allows defining abstract base classes _aka_ ABC.

A class is abstract if:
- its metaclass is `ABCMeta`
- at least one of abstract methods don't have an implementation

In [88]:
import abc


class Iterable(metaclass=abc.ABCMeta):
    @abc.abstractclassmethod
    def __iter__(self):
        pass

In [89]:
class Something(Iterable):
    pass

In [90]:
Something()

TypeError: Can't instantiate abstract class Something with abstract methods __iter__

💡 [collections.abc — Abstract Base Classes for Containers](https://docs.python.org/3/library/collections.abc.html)